In [11]:
#[packages]
#cameo, escher, pandas, rise
#[requires]
#python_version = "3.8"
#postBuild: jupyter labextension install jupyterlab-plotly@4.14.3

In [5]:
import cobra
cobra_config = cobra.Configuration()
cobra_config.solver ="glpk_exact"

abc_model = cobra.Model("ABC_model")
A = cobra.Metabolite("A", compartment="c")
B = cobra.Metabolite("B", compartment="c")
C = cobra.Metabolite("C", compartment="c")
D = cobra.Metabolite("D", compartment="c")
E = cobra.Metabolite("E", compartment="c")
F = cobra.Metabolite("F", compartment="c")

abc_model.add_metabolites([A,B,C,D,E,F])

R_1 = cobra.Reaction("R_1")
R_2 = cobra.Reaction("R_2")
R_3 = cobra.Reaction("R_3")
R_4 = cobra.Reaction("R_4")
R_5 = cobra.Reaction("R_5")
R_6 = cobra.Reaction("R_6")
R_7 = cobra.Reaction("R_7")
R_8 = cobra.Reaction("R_8")
R_9 = cobra.Reaction("R_9")

abc_model.add_reactions([R_1, R_2, R_3, R_4, R_5, R_6, R_7, R_8, R_9])

R_1.build_reaction_from_string("--> A")
R_2.build_reaction_from_string("A --> B")
R_3.build_reaction_from_string("A --> C")
R_4.build_reaction_from_string("B + E --> 2 D")
R_5.build_reaction_from_string("--> E")
R_6.build_reaction_from_string("2 B --> C + F")
R_7.build_reaction_from_string("C --> D")
R_8.build_reaction_from_string("D -->")
R_9.build_reaction_from_string("F -->")

cobra.io.save_json_model(abc_model, "ABC_model.json")
cobra.util.array.create_stoichiometric_matrix(abc_model, array_type="DataFrame").astype(int)

,R_1,R_2,R_3,R_4,R_5,R_6,R_7,R_8,R_9
A,1,-1,-1,0,0,0,0,0,0
B,0,1,0,-1,0,-2,0,0,0
C,0,0,1,0,0,1,-1,0,0
D,0,0,0,2,0,0,1,-1,0
E,0,0,0,-1,1,0,0,0,0
F,0,0,0,0,0,1,0,0,-1


In [6]:
abc_model = cobra.io.load_json_model("ABC_model.json")  #balanced steady state
abc_model.objective = "R_8" #cellular objective (growth)
for rxn in abc_model.reactions:
    rxn.lower_bound = 0 #irreversible reaction, cannot be <0
abc_model.reactions.R_1.upper_bound = 10 #uptake rate 10mmol/gDW/hr

optimal_growth_rate = abc_model.optimize() #find fluxes that maximize growth rate
optimal_growth_rate

,fluxes,reduced_costs
R_1,10.0,4.0
R_2,10.0,0.0
R_3,0.0,-2.0
R_4,10.0,0.0
R_5,10.0,0.0
R_6,0.0,-6.0
R_7,0.0,0.0
R_8,20.0,0.0
R_9,0.0,0.0


In [7]:
#list of numbers may not be helpful, so we can visualize the fluxes

In [10]:
import escher #package developed by inventors of flux balance analysis
#ERROR for cannot import soft_unicode from markup_safe: downgrade to MarkupSafe==2.1.1 and Jinja2==3.1.2

ImportError: cannot import name 'soft_unicode' from 'markupsafe' (C:\Users\lint730\AppData\Local\anaconda3\Lib\site-packages\markupsafe\__init__.py)

In [9]:
abc_model = cobra.io.load_json_model('ABC_model.json')
abc_model.objective = abc_model.reactions.R_9
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10
optimal_bioproduct_yield = abc_model.optimize()                 
display(optimal_bioproduct_yield)

reaction_scale = [ { 'type': 'min',  'color': '#c8c8c8', 'size': 12 },
                   { 'type': 'mean', 'color': '#9696ff', 'size': 20 },
                   { 'type': 'max',  'color': '#ff0000', 'size': 25 } ]
escher.Builder( map_json       = 'ABC_map.json.loads',
                model          = abc_model,
                reaction_data  = optimal_growth_rate.fluxes.to_dict(),
                reaction_scale = reaction_scale
              )

,fluxes,reduced_costs
R_1,10.0,1.0
R_2,10.0,0.0
R_3,0.0,-1.0
R_4,0.0,-1.0
R_5,0.0,0.0
R_6,5.0,0.0
R_7,5.0,0.0
R_8,5.0,0.0
R_9,5.0,0.0


NameError: name 'escher' is not defined

In [1]:
abc_model = cobra.io.load_json_model('ABC_model.json')
abc_model.objective = abc_model.reactions.R_9
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10

optimal_bioproduct_yield = abc_model.optimize()
display(optimal_bioproduct_yield)
escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = optimal_bioproduct_yield.fluxes.to_dict(),
                reaction_scale = reaction_scale
              )
#optimal solution with objective value 5.0, halfing the input flux of 10

NameError: name 'cobra' is not defined

In [4]:
abc_model = cobra.io.load_json_model('ABC_model.json') #stoichiometric matrix loaded
abc_model.objective = R_8                              #cellular objective (growth)
for rxn in abc_model.reactions:                        
    rxn.lower_bound = 0                                #irreversible reactions
abc_model.reactions.R_3.upper_bound = 0                #Uptake rate = 10mmol/hr
abc_model.reactions.R_5.upper_bound = 5                #An-E-robic environmental condition

environment_solution = abc_model.optimize()            #Effect of altering envrionment on bioproduct yield
display(environment_solution)

escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale,
              )

NameError: name 'cobra' is not defined

In [11]:
#Product:the optimal flux in the absence of E (r5 = 0) is to convert A to D from R1 -> R8 pathway. Optimal growth rate grew slower,
#but did not produce bioproduct, non-growth product solution

In [1]:
#knock out gene that enables the A -> D pathway to force growth to optimize growth and bioproducts
abc_model = cobra.io.load_json_model('ABC_model.json')
abc_model.objective = R_8
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10
abc_model.reactions.R_3.upper_bound = 0                  #Addition of genetic perturbation to prevent taking A to D pathway
abc_model.reactions.R_5.upper_bound = 0

environment_and_ko_solution = abc_model.optimize()       #Find fluxes that balance steady-state, optimizing growht and bioproduct

display(environment_and_ko_solution)
escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_and_ko_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale
              )

NameError: name 'cobra' is not defined

In [2]:
#reaction takes third pathway from a-> b > F > D
#How about productivity? (product of growth rate and yield) after knocking out R3 and restricting to a an-E-robic condition

In [3]:
#trying to improve aerobic conditions to see how that affects the trade off of product and yield. Maximize R8 and give organism
#more oxygen
abc_model = cobra.io.load_json_model('ABC_model.json')   #Stoichiometric matrix loaded
abc_model.objective = R_8                                #Cellular objective (growth)
for rxn in abc_model.reactions:
    rxn.lower_bound = 0                                  #Irreversible reaction
abc_model.reactions.R_1.upper_bound = 10                 #Uptake 10mmol/hour
abc_model.reactions.R_3.upper_bound = 0                  #Addition of genetic perturbation to prevent taking A to D pathway
abc_model.reactions.R_5.upper_bound = 5                  #An-E-robic environmental condition

environment_and_ko_solution = abc_model.optimize()       #Find fluxes that balance steady-state, optimizing growht and bioproduct

display(environment_and_ko_solution)
escher.Builder( map_json       ='ABC_map.json',
                model          = abc_model,
                reaction_data  = environment_and_ko_solution.fluxes.to_dict(), 
                reaction_scale = reaction_scale
              )
#resulting color temp scale on graph = larger the number, more red the pathway
#producing growth at rate of 12.5 (producing more than under anerobic conditions) but bioproduct is less (5 > 2.5). But
#Product of 12.5x2.5 is > product of 5 x 5

NameError: name 'cobra' is not defined

In [4]:
#Visualising Tr/O bn growth rate and bioproduct yield w Production Envelope

In [5]:
#Production envelope helps extend intuition understanding tr/o to genome-
#scale models
#growth rate previously an objective, now it's a constraint (forcing growth
#at specific growth rate, maximizing engineering objective--bioproduct)
from cameo.flux_analysis import phenotypic_phase_plane as ppp
from cameo.visualization.plotting.with_plotly import PlotlyPlotter
from cameo.visualization import plotting

abc_model = cobra.io.load_json_model('ABC_model.json')
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10

production_envelope = ppp( abc_model,
                              variables=[abc_model.reactions.R_8],     #Grwoth rate <=i
                              objective=abc_model.reactions.R_9,
                              points=21)     #Engineering objective (bioproduct)

result_df = production_envelope.data_frame.rename( columns  = dict(
                                                            R_8 = 'growth_rate',
                                          objective_upper_bound = 'bioproduct_maximum',
                                          objective_lower_bound = 'bioproduct_minimum'))
plotter = PlotlyPlotter()
production_envelope.plot(plotter, 
                         title='Production envelope between bioproduct yield and growth rate for WT ABC model',
                         points=[(5,5), (20,0),(10,0)])
#Result is triangular 'envelope' that describes how much biopro. you can generate
#(optimally) given a particular growth rate

ModuleNotFoundError: No module named 'cameo'

In [6]:
from cameo.flux_analysis import phenotypic_phase_plane as production_envelope

abc_model = cobra.io.load_json_model('ABC_model.json')
for rxn in abc_model.reactions:
    rxn.lower_bound = 0
abc_model.reactions.R_1.upper_bound = 10
abc_model.reactions.R_3.upper_bound = 0
abc_model.reactions.R_5.upper_bound = 5            #low E-robic environmental addition

result = production_envelope( abc_model,
                              variables=[abc_model.reactions.R_8],
                              objective=abc_model.reactions.R_9,
                              points=21 )    

result_df = result.data_frame.rename(columns=dict(R_8='growth_rate',
                                objective_upper_bound='bioproduct_maximum',
                                objective_lower_bound='bioproduct_minimum'))
result.plot(plotter,title='Gene knockout solution on the production envelope' #for attempt
            #to grow and optimally yield
            #optimal growth rate now achieves a nonzero product
#^solution of gene knockout is (in theory) evolutionarily stable

SyntaxError: unexpected EOF while parsing (3460548288.py, line 20)

In [7]:
#Flux Variability analysis for pre-processing estimate of bounds

In [8]:
import escher
import cobra
from cameo import flux_variability_analysis

abc_model =  cobra.io.load_json_model('ABC_model.json') 
abc_model.reactions.R_1.upper_bound=10
fva_result = flux_variability_analysis(abc_model)


fva_result.data_frame
fva_result.plot(plotter,index=fva_result.data_frame.index, height=600)
#You end up getting bounds much tighter than original rounds using FVA--
#shows other constraints induced by the first constraint

ModuleNotFoundError: No module named 'escher'

In [9]:
#FVA loops through rxns in network, solving for MAX flux and solving for MIN flux
#associated with that reaction:

import pandas as pd

def fva( model ):
    fva = {'minimum':{},
           'maximum':{}}
    for reaction in model.reactions:
        with model:
            model.objective = {reaction: -1}
            fva['minimum'][reaction.id] = model.slim_optimize()
        with model:
            model.objective = {reaction: 1}
            fva['maximum'][reaction.id] =  model.slim_optimize()
    return pd.DataFrame(fva)[['minimum','maximum']].

SyntaxError: invalid syntax (427761037.py, line 16)

In [10]:
#What can you do with FVA

In [ ]:
import escher
import cobra
from cameo import flux_variability_analysis

abc_model =  cobra.io.load_json_model('ABC_model.json') 
abc_model.reactions.R_1.upper_bound=10

abc_fva = fva(abc_model)
display(abc_fva)
escher.Builder(map_json='ABC_map.json',
               model=abc_model,
               reaction_data=abc_fva['maximum'].to_dict(),
              )
#reactions now all have constraints induced by a single constraint (R1)
#useful for modeling expression data ie how big can this reaction get?